# Sequential Chain

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

llm = AzureChatOpenAI(
    model_name = "gpt-4o",
    temperature=0.8,
    max_tokens=512
)

template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个200字左右的介绍。
花名: {name}
颜色: {color}
植物学家: 这是关于上述花的介绍:"""

prompt = PromptTemplate.from_template(template=template)
response = llm.invoke(prompt.format_prompt(name="玫瑰", color="黑色"))

In [2]:
parser = StrOutputParser(output_key="introduction")

print(parser.parse(response.content))

黑色玫瑰，这种花朵往往被赋予神秘、优雅和热情的象征意义。尽管在自然界中没有真正意义上的黑色玫瑰，其实它们通常是深红色或深紫色，通过特定的光线和背景显现为黑色。然而，通过精细的园艺技术和选择性的培育，园艺师们能创造出接近黑色的玫瑰品种，如‘黑巴卡拉’（Black Baccara）和‘午夜玫瑰’（Midnight Rose）。

黑色玫瑰的外观令人惊艳，花瓣光滑且富有光泽，往往带有丝绸般的质感。它们多用于装饰、花艺设计和特别的场合，如婚礼、纪念活动等，通常象征着新生和重生或终结和告别。

在种植方面，黑色玫瑰需要充足的日照和适宜的温度，最好在肥沃、排水良好的土壤中生长。定期的修剪和病虫害防治也是确保其健康生长的关键。这种花不仅美丽，还能为花园增添一份独特的魅力，是花卉爱好者的珍贵品种。


In [3]:
introduction_chain = prompt | llm | parser
response = introduction_chain.invoke(input={"name":"玫瑰", "color":"黑色"})
print(type(response), response)

<class 'str'> 黑色玫瑰是一种罕见且神秘的花卉，具有深沉而独特的美感。尽管在自然界中并不存在真正的黑色玫瑰，这种花通常是通过深紫色或深红色品种的玫瑰在特定光线下呈现出的效果。黑色玫瑰象征着许多独特的意涵，包括神秘、告别、新的开始和复仇。它们可以在园艺和花艺设计中为布置增添一丝独特的氛围和戏剧性。

从植物学的角度来看，玫瑰属于蔷薇科，是一种多刺的灌木，具有复杂而美丽的花瓣结构。黑色玫瑰的引种和培育需要特别的照料，以确保其颜色的深度和品质。通常，这种花卉会在温度较低和光线较少的环境中培育，以增强其深色调。

黑色玫瑰在艺术和文学中也有着重要的地位，常被用来表达深沉的感情和复杂的情感状态。它们不仅是园艺爱好者的珍贵收藏，同时也是表达特别信息的优雅方式。无论是作为一个特别的礼物，还是作为一件艺术品，黑色玫瑰都能带来一种无法忽视的魅力。


In [4]:
template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个200字左右的介绍。
花名: {name}
颜色: {color}
植物学家: 这是关于上述花的介绍:"""

prompt = PromptTemplate(
    input_variables=["name", "color"],
    template=template
)
introduction_chain = prompt | llm | {"introduction": StrOutputParser()}


template = """
你是一位鲜花评论家。给定一种花的介绍，你需要为这种花写一篇200字左右的评论。
鲜花介绍:
{introduction}
花评人对上述花的评论:"""
prompt2 = PromptTemplate(
    input_variables=["introduction"],
    template=template
)
review_chain = prompt2 | llm | {"review": StrOutputParser()}

template = """
你是一家花店的社交媒体经理。给定一种花的介绍和评论，你需要为这种花写一篇社交媒体的帖子，300字左右。
鲜花介绍:
{introduction}
花评人对上述花的评论:
{review}
社交媒体帖子:
"""
prompt3 = PromptTemplate(
    input_variables=["introduction", "review"],
    template=template
)
social_post_chain = prompt3 | llm | {"social_post_text": StrOutputParser()}




In [5]:
# Not work
# overall_chain = SequentialChain(
#     chains=[introduction_chain, review_chain, social_post_chain],
#     input_variables=["name", "color"],
#     output_variables=["introduction", "review", "social_post_text"],
#     # verbose=True
# )

overall_chain = introduction_chain | review_chain
result = overall_chain.invoke({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)


{'review': '黑色玫瑰，这种几近神话般的花卉，仿佛是大自然与人类智慧共同合作的杰作。虽然在自然界中未曾真正存在，但通过染色技术或特定的生长条件，黑色玫瑰的深紫色或深红色外观成功地营造出一种近似黑色的奇幻效果。它的独特颜色不仅在视觉上引人入胜，更在象征意义上充满了神秘与权力，庄重与高贵，甚至有时还能表达悲伤或告别的情感。\n\n作为蔷薇科的一员，黑色玫瑰继承了玫瑰家族复杂而丰富的花瓣结构，芳香四溢，令人陶醉。其生长条件虽然要求充足的阳光和良好的排水，但适应性极强，让花园爱好者的栽培工作变得更为简便。值得一提的是，土壤的pH值应保持在6.0至7.0之间，这一点对于获得最佳花色至关重要。定期的修剪和适当的施肥则是保证其茁壮成长的关键。\n\n黑色玫瑰无疑在观赏价值和象征意义上达到了一个新的高度。这种花卉不仅是一种视觉上的享受，更是情感和文化的载体。其罕见和魅力让每一个见到它的人都为之惊叹，仿佛在这朵花中看到了大自然和人类共同追求的美丽与神秘。黑色玫瑰不仅仅是一朵花，更是一种象征、一种情感的表达，一种高贵与庄重的体现。'}


In [6]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

introduction_chain = prompt | llm | StrOutputParser() | {"introduction": RunnablePassthrough()}
review_chain = prompt2 | llm | {"review": StrOutputParser()} | {"review": RunnablePassthrough()}
social_post_chain = prompt3 | llm | {"social_post_text": StrOutputParser()}

overall_chain2 = introduction_chain | {"introduction": itemgetter("introduction"), "review": review_chain} | social_post_chain

result = overall_chain2.invoke({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)

{'social_post_text': '🌹✨黑色玫瑰：深沉之美与神秘的象征✨🌹\n\n你是否曾经被一种花卉深深吸引，不仅因为它的美丽，更因为它所传达的深远意义？让我们一同走进黑色玫瑰的世界，这种独特的花卉以其神秘的深紫色或深红色花瓣，成为视觉和情感传递的奇观。\n\n黑色玫瑰，尽管在自然界中并不存在真正的黑色，但通过现代育种技术和染色方法，我们得以欣赏到它那近似黑色的迷人色彩。这种花卉象征着神秘、独立、变革和新生，常常被用来表达深沉的情感和无尽的尊重。无论是在园艺中作为观赏植物，还是在文学、艺术和电影中作为象征性元素，黑色玫瑰都发挥着不可忽视的作用。\n\n黑色玫瑰不仅仅是一种美丽的花卉，更是一种文化和情感的象征。它们深邃的颜色和丰富的内涵，使人们在欣赏其美丽的同时，也感受到一种无言的力量和深沉的美。作为落叶灌木的玫瑰，具有典型的直立茎和多刺的枝条，这些特征赋予了它们顽强的生命力和独特的美感。它们在阳光充足、排水良好的环境中生长良好，但需要注意防治蚜虫和白粉病等病虫害。\n\n总的来说，黑色玫瑰不仅是视觉上的奇观，更是一种情感和思想的传递媒介。它们的存在，不仅仅是为了装点生活，更是为了激发思考和情感的共鸣。让我们一起用这深沉的花朵，表达那些不易言表的深情和敬意。🌹❤️\n\n#黑色玫瑰 #深沉之美 #情感传递 #花卉象征 #神秘与独立 #文化与艺术 #园艺爱好者 #花店推荐 #心灵共鸣'}


简化上面的写法：

In [7]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

introduction_chain = prompt | llm | {"introduction": RunnablePassthrough()}
review_chain = prompt2 | llm | {"review": RunnablePassthrough()}
social_post_chain = prompt3 | llm | {"social_post_text": StrOutputParser()}

overall_chain2 = introduction_chain | {"introduction": itemgetter("introduction"), "review": review_chain} | social_post_chain

result = overall_chain2.invoke({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)

{'social_post_text': '🌹✨神秘的黑色玫瑰：自然与艺术的结合✨🌹\n\n在众多玫瑰品种中，黑色玫瑰无疑是最为神秘和引人入胜的存在。虽然纯黑色玫瑰在自然界中并不存在，但通过巧妙的培育和光线效果，我们得以欣赏到近乎黑色的绚丽花朵。这些玫瑰，多为深紫色或深红色品种，如著名的“巴卡拉”变种，以其深邃的色调和独特的美感而著称。\n\n🌺 黑色玫瑰不仅是一种视觉上的享受，它们还在文学和文化中承载着丰富的象征意义。神秘、死亡、复仇和新生，这些深刻且复杂的主题，都能在黑色玫瑰中找到影射。它们仿佛提醒着我们，生命中有着多重维度和人类情感的复杂性。\n\n🌱 从园艺学的角度来看，黑色玫瑰的栽培条件相当苛刻，需要在土壤肥沃、光照充足的环境中才能茁壮成长。这无疑也体现了园艺学家和植物学家们的智慧和辛勤努力，他们通过科学与艺术的结合，不断探索自然界的极限，为我们带来这样的奇迹。\n\n🔍 总体而言，黑色玫瑰不仅是一种视觉上的享受，更是一种精神上的启迪。它们独特的颜色和深刻的象征意义，使其在众多花卉中独树一帜，成为经典与创新的完美结合。无论是作为观赏植物还是文化符号，黑色玫瑰都展现出非凡的魅力和价值。\n\n🌹✨快来我们的花店，一同欣赏这份来自自然与艺术的神秘礼物吧！✨🌹\n\n#黑色玫瑰 #神秘之美 #园艺奇迹 #花店 #爱情与美丽 #文化象征'}


In [8]:
introduction_chain = prompt | llm | {"introduction": StrOutputParser()}
review_chain = prompt2 | llm | {"review": StrOutputParser()}
social_post_chain = prompt3 | llm

overall_chain3 = introduction_chain | {"introduction": itemgetter("introduction"), "review": review_chain} | social_post_chain

result = overall_chain3.invoke({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)

content='🌹✨揭开黑色玫瑰的神秘面纱✨🌹\n\n你是否曾被一种花的神秘与独特所深深吸引？今天，我们要为大家揭开一种罕见而迷人的花卉——黑色玫瑰的神秘面纱。尽管自然界中并不存在真正的黑色玫瑰，但通过特殊的培育和光线条件，这些深红色或深紫色玫瑰呈现出近黑的视觉效果，散发出一种独特的魅力。\n\n黑色玫瑰不仅仅是一朵花，它还是文化和情感的象征。在文学与艺术作品中，黑色玫瑰常常代表神秘、死亡与再生，告别与新生。它们那深沉的美感和复杂的象征意义，使其成为许多创作灵感的源泉。尤其在哥特文化中，黑色玫瑰更是暗黑美学的核心元素，象征着一种深邃的、略带忧郁的浪漫。\n\n从植物学的角度来看，黑色玫瑰属于蔷薇科，多年生灌木，花瓣层层叠叠，散发出浓郁的香气。它们适应性强，但对土壤和光照有特定要求。特别是黑色玫瑰的培育，需要精心选择品种和严格控制生长环境，以确保其颜色的最佳展示。当花瓣层层绽放，深色的花朵散发出浓郁的香气时，那种美丽与庄重无疑让人心醉神迷。\n\n黑色玫瑰不仅是一种视觉的享受，更是一种情感的体验和文化的象征。它们的存在提醒我们，美丽可以有多种形式，而每一种形式都值得我们去探索和珍视。黑色玫瑰以其独特的稀有性和深沉的美感，成为花卉界一颗耀眼的明星，令人陶醉、流连忘返。\n\n#黑色玫瑰 #神秘之花 #浪漫与美丽 #花卉艺术 #哥特美学 #爱情象征' response_metadata={'token_usage': {'completion_tokens': 509, 'prompt_tokens': 838, 'total_tokens': 1347}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence'

In [9]:
print(result.content)

🌹✨揭开黑色玫瑰的神秘面纱✨🌹

你是否曾被一种花的神秘与独特所深深吸引？今天，我们要为大家揭开一种罕见而迷人的花卉——黑色玫瑰的神秘面纱。尽管自然界中并不存在真正的黑色玫瑰，但通过特殊的培育和光线条件，这些深红色或深紫色玫瑰呈现出近黑的视觉效果，散发出一种独特的魅力。

黑色玫瑰不仅仅是一朵花，它还是文化和情感的象征。在文学与艺术作品中，黑色玫瑰常常代表神秘、死亡与再生，告别与新生。它们那深沉的美感和复杂的象征意义，使其成为许多创作灵感的源泉。尤其在哥特文化中，黑色玫瑰更是暗黑美学的核心元素，象征着一种深邃的、略带忧郁的浪漫。

从植物学的角度来看，黑色玫瑰属于蔷薇科，多年生灌木，花瓣层层叠叠，散发出浓郁的香气。它们适应性强，但对土壤和光照有特定要求。特别是黑色玫瑰的培育，需要精心选择品种和严格控制生长环境，以确保其颜色的最佳展示。当花瓣层层绽放，深色的花朵散发出浓郁的香气时，那种美丽与庄重无疑让人心醉神迷。

黑色玫瑰不仅是一种视觉的享受，更是一种情感的体验和文化的象征。它们的存在提醒我们，美丽可以有多种形式，而每一种形式都值得我们去探索和珍视。黑色玫瑰以其独特的稀有性和深沉的美感，成为花卉界一颗耀眼的明星，令人陶醉、流连忘返。

#黑色玫瑰 #神秘之花 #浪漫与美丽 #花卉艺术 #哥特美学 #爱情象征
